# Foreign Key Processing

In [1]:
# ! py -m pip install networkx
# ! py -m pip install thefuzz

In [1]:
import pandas as pd
import networkx as nx
from thefuzz import fuzz
from thefuzz import process
import numpy as np

In [3]:
path = 'C:\\Users\\mounsl\\OneDrive - Idaho National Laboratory\\Documents - GRP-Agree Data Assessment Team\\Security\\3. Data and Modeling\\Data Dictionary\\IFACTS\\' #path to where files for the database are kept
database = 'IFACTS'
date = '20240515'

In [4]:
raw_FKs = pd.read_excel(path+database+'_FKs_'+date+'.xlsx')

In [7]:
raw_FKs

,SchemaName,TableName,ReferencedTable,ColumnName,FK_name,ReferencedColumn
0,dbo,aspnet_Membership,aspnet_Applications,ApplicationId,FK__aspnet_Me__Appli__1FD8A9E3,ApplicationId
1,dbo,aspnet_Membership,aspnet_Users,UserId,FK__aspnet_Me__UserI__20CCCE1C,UserId
2,dbo,aspnet_Paths,aspnet_Applications,ApplicationId,FK__aspnet_Pa__Appli__51700577,ApplicationId
3,dbo,aspnet_PersonalizationAllUsers,aspnet_Paths,PathId,FK__aspnet_Pe__PathI__5728DECD,PathId
4,dbo,aspnet_PersonalizationPerUser,aspnet_Paths,PathId,FK__aspnet_Pe__PathI__5AF96FB1,PathId
...,...,...,...,...,...,...
111,dbo,datUserReportCol,datUserReport,UserReport_ID,FK_datUserReportCol_datUserReport,UserReport_ID
112,dbo,datUserReportUdf,datReportUdf,ReportUdf_ID,FK_datUserReportUdf_datReportUdf,ReportUdf_ID
113,dbo,datUserReportUdf,datUserReport,UserReport_ID,FK_datUserReportUdf_datUserReport,UserReport_ID
114,dbo,luCountry,luCountryType,CountryType_ID,FK_luCountry_luCountryType,CountryType_ID


In [175]:
raw_FKs[['TableName','ColumnName','ReferencedColumn','ReferencedTable']].head(50)

,TableName,ColumnName,ReferencedColumn,ReferencedTable
0,aspnet_Membership,ApplicationId,ApplicationId,aspnet_Applications
1,aspnet_Membership,UserId,UserId,aspnet_Users
2,aspnet_Paths,ApplicationId,ApplicationId,aspnet_Applications
3,aspnet_PersonalizationAllUsers,PathId,PathId,aspnet_Paths
4,aspnet_PersonalizationPerUser,PathId,PathId,aspnet_Paths
5,aspnet_PersonalizationPerUser,UserId,UserId,aspnet_Users
6,aspnet_Profile,UserId,UserId,aspnet_Users
7,aspnet_Roles,ApplicationId,ApplicationId,aspnet_Applications
8,aspnet_Users,ApplicationId,ApplicationId,aspnet_Applications
9,aspnet_UsersInRoles,RoleId,RoleId,aspnet_Roles


## Analysis with NetworkX

Documentation: https://networkx.org/documentation/stable/reference/index.html

In [8]:
edgelist = raw_FKs[['TableName', 'ReferencedTable']]
edgelist.to_csv(database+'_edgelist.csv', index=False, header=False)
fk_graph = nx.read_edgelist(database+'_edgelist.csv', delimiter=',',create_using=nx.DiGraph)
fk_graph.number_of_nodes()
fk_graph.number_of_edges()
nx.is_directed_acyclic_graph(fk_graph)
nx.is_weakly_connected(fk_graph)
nx.number_weakly_connected_components(fk_graph)
for c in nx.weakly_connected_components(fk_graph):
    print(len(c), ':', c)

## Possible Uninforced Foreign Keys with Fuzzy Matching


In [18]:
raw_cols = pd.read_excel(path+'Working\\'+database+'_RawTableChars.xlsx', usecols=['TableName', 'ColumnName'])

In [19]:
raw_cols

,TableName,ColumnName
0,aspnet_Applications,ApplicationId
1,aspnet_Applications,ApplicationName
2,aspnet_Applications,Description
3,aspnet_Applications,LoweredApplicationName
4,aspnet_Membership,ApplicationId
...,...,...
1400,sysdiagrams,definition
1401,sysdiagrams,diagram_id
1402,sysdiagrams,name
1403,sysdiagrams,principal_id


In [158]:
def extract_matches(cols, existing_relationships):
    cols["match results"] = cols['ColumnName'].apply(lambda x: process.extract(x, cols['ColumnName'], scorer=fuzz.ratio))
    new_rows = []
    for index, row in cols.iterrows():
        for result in row['match results']:
            new_row = {'TableName':row['TableName'],
                    'ColumnName':row['ColumnName'],
                    'MatchColumn':result[0],
                    'MatchRatio':result[1],
                    'MatchTable':cols.iloc[result[2]]['TableName']}
            new_rows.append(new_row)
    new_df = pd.DataFrame(new_rows, columns=['TableName','ColumnName','MatchColumn','MatchRatio','MatchTable'])
    new_df = new_df[(new_df['TableName'] != new_df['MatchTable'])]
    new_df['combo'] = new_df['TableName']+','+new_df['ColumnName']+' to '+new_df['MatchTable']+','+new_df['MatchColumn']
    new_df = new_df[(~new_df['combo'].isin(existing_relationships))&(new_df['MatchRatio']>80)].drop(columns=['combo'])
    return new_df[['TableName','ColumnName','MatchColumn','MatchTable','MatchRatio']]
        
def search_term(df, term, regex=False):
    results_df = df[(df['MatchName'].str.contains(term,case=False,regex=regex))|(df['ColumnName'].str.contains(term,case=False, regex=regex))]
    return results_df

In [159]:
fks = raw_FKs

In [160]:
### XXX True deduping needs to disregard direction of relationship: merge TableName and ReferencedTable ColumnName and ReferencedColumn THEN dedup
fks['dir1'] = fks['TableName']+','+fks['ColumnName']+' to '+fks['ReferencedTable']+','+fks['ReferencedColumn']
fks['dir2'] = fks['ReferencedTable']+','+fks['ReferencedColumn']+' to '+fks['TableName']+','+fks['ColumnName']


In [161]:
all_relationships = pd.concat([fks['dir1'],fks['dir2']])

In [162]:
all_relationships

0      aspnet_Membership,ApplicationId to aspnet_Appl...
1        aspnet_Membership,UserId to aspnet_Users,UserId
2      aspnet_Paths,ApplicationId to aspnet_Applicati...
3      aspnet_PersonalizationAllUsers,PathId to aspne...
4      aspnet_PersonalizationPerUser,PathId to aspnet...
                             ...                        
111    datUserReport,UserReport_ID to datUserReportCo...
112    datReportUdf,ReportUdf_ID to datUserReportUdf,...
113    datUserReport,UserReport_ID to datUserReportUd...
114    luCountryType,CountryType_ID to luCountry,Coun...
115    luSoftwareType,SoftwareType_ID to luSoftware,S...
Length: 232, dtype: object

In [163]:
len(all_relationships)

232

In [164]:
match_df = extract_matches(raw_cols,all_relationships)


### Custom Munging

In [165]:
excluded_tables = ['aspnet_PersonalizationAllUsers', 'aspnet_PersonalizationPerUser', 'aspnet_Paths', 'aspnet_Users', 'aspnet_Profile', 'aspnet_Roles', 'aspnet_Applications', 'aspnet_UsersInRoles', 'aspnet_Membership','datReportTbl', 'datReportTblCol', 'luReportColType', 'datUserReportUdf', 'datUserReportCol', 'datUserReport', 'datReportUdf']

In [166]:
match_df = match_df[(~match_df['TableName'].isin(excluded_tables))&(~match_df['MatchTable'].isin(excluded_tables))]

In [167]:
match_df = match_df[(~match_df['TableName'].str.contains('LOG'))&(~match_df['MatchTable'].str.contains('LOG'))]

In [168]:
match_df = match_df[(~match_df['TableName'].str.startswith('br'))&(~match_df['MatchTable'].str.startswith('br'))]

In [169]:
match_df = match_df[(~match_df['TableName'].str.startswith('lu'))]

In [170]:
match_df = match_df[(~match_df['TableName'].str.contains('REV'))&(~match_df['MatchTable'].str.contains('REV'))]

In [173]:
match_df.to_csv('IFACTS_FK_processing_test.csv',index=False)
#match_df.to_excel(path+'Working\\'+database+'_Possible_FKs.xlsx',index=False)

In [172]:
raw_FKs[(raw_FKs['TableName']=='aspnet_Applications')|(raw_FKs['ReferencedTable']=='aspnet_Applications')][['TableName','ReferencedTable','ColumnName','ReferencedColumn']]

,TableName,ReferencedTable,ColumnName,ReferencedColumn
0,aspnet_Membership,aspnet_Applications,ApplicationId,ApplicationId
2,aspnet_Paths,aspnet_Applications,ApplicationId,ApplicationId
7,aspnet_Roles,aspnet_Applications,ApplicationId,ApplicationId
8,aspnet_Users,aspnet_Applications,ApplicationId,ApplicationId


## Possible External Relationships

In [ ]:
# SNumber
# Dates
# Facilities/Buildings
#

### Custom Analysis

# TRAIN Analysis

In [14]:
train = pd.read_excel('C:\\Users\\mounsl\\OneDrive - Idaho National Laboratory\\Documents - GRP-Agree Data Assessment Team\\Training\\3. Data & Modeling\\Data Dictionary\\Working\\TRAIN_PKFK.xlsx')

In [16]:
train#[train['TABLENAME']]

,TABLENAME,COLNAME,Referenced Table,Referenced Column,PK_ordinal_position
0,ADMINISTRATOR_ORG,CLASS_ADMIN,NaN,NaN,NaN
1,ADMINISTRATOR_ORG,CONTRACT_ID,PRIME_CONTRACTOR,CONTRACT_ID,NaN
2,ADMINISTRATOR_ORG,EMP_SSN_NUM,EMPLOYEE,EMP_SSN_NUM,NaN
3,ADMINISTRATOR_ORG,ORG_CODE,ORG,ORG_CODE,NaN
4,ADMINISTRATOR_ORG,ORG_ENROLLER,NaN,NaN,NaN
...,...,...,...,...,...
415,READING,RE_READ_INTVL_MTH,NaN,NaN,NaN
416,READING,SRCE_REQ_DESC,NaN,NaN,NaN
417,READING,TRAIN_DUR_HRS,NaN,NaN,NaN
418,READING,UPDATE_DATE,NaN,NaN,NaN


In [17]:
no_reference = train[(~train['Referenced Table'].isin(train['TABLENAME'].unique().tolist()))&(~train['Referenced Table'].isna())]

In [18]:
no_reference

,TABLENAME,COLNAME,Referenced Table,Referenced Column,PK_ordinal_position
1,ADMINISTRATOR_ORG,CONTRACT_ID,PRIME_CONTRACTOR,CONTRACT_ID,NaN
18,ATTEND_TEST,INSTR_TYPE_CODE,INSTR_TYPE,INSTR_TYPE_CODE,3.0
40,CLASS,INSTR_TYPE_CODE,INSTR_TYPE,INSTR_TYPE_CODE,NaN
47,CLASS,VEND_CODE,VEND,VEND_CODE,NaN
89,COURSE,CONTRACT_ID,PRIME_CONTRACTOR,CONTRACT_ID,NaN
101,COURSE,FACILITY_CODE,FACILITY,FACILITY_CODE,NaN
104,COURSE,INSTR_TYPE_CODE,INSTRUCTION_TYPE,INSTRUCTION_TYPE_CODE,3.0
111,COURSE,REPORT_CAT,TRAIN_CAT,TRAIN_CAT_CODE,NaN
121,COURSE,TRM_CAT,TRM_CAT,TRM_CAT_ID,NaN
132,COURSE_WAITING_LIST,INSTR_TYPE_CODE,INSTRUCTION_TYPE,INSTRUCTION_TYPE_CODE,2.0


In [20]:
no_reference['Referenced Table'].unique().tolist()

['PRIME_CONTRACTOR',
 'INSTR_TYPE',
 'VEND',
 'FACILITY',
 'INSTRUCTION_TYPE',
 'TRAIN_CAT',
 'TRM_CAT',
 'ADMIN_DOC',
 'AREAS',
 'COMPANY',
 'JC_FUNCTION',
 'LOCATION',
 'JC_POSITION',
 'PROGRAM']